Scroll down to see what is happening

In [1]:
import pandas as pd 
import numpy as np
import csv
import re
from thefuzz import process


In [2]:
df = pd.read_excel("https://www.sharkattackfile.net/spreadsheets/GSAF5.xls")


In [3]:
df.drop(["pdf", "href formula", "Source", "href", "Case Number", "Case Number.1", 'original order', 'Unnamed: 21', 'Unnamed: 22'], axis=1, inplace=True)

In [4]:
# cleaning Type
df.dropna(subset=["Type"], inplace=True)
df["Type"] = df["Type"].str.strip().str.title()
df["Type"] = df["Type"].replace({"Unverified": "Unknown", "Under Investigation": "Unknown", "Unconfirmed": "Unknown", "?": "Unknown", "Questionable": "Unknown", "Invalid": "Unknown", "Watercraft": "Unprovoked-like", "Sea Disaster": "Unprovoked-like", "Boat": "Unprovoked-like"})
# unproked-like means they did not mean to porvoke the shark but the shark got provoked by the noise or the movement of the boat ext.

In [5]:
#Provoked value

def full_type(value):
    if value == "Unprovoked":
        return "Unprovoked"
    elif value == "unprovoked":
        return "Unprovoked"        
    elif value == "Provoked":
        return "Provoked"
    elif value == ' Provoked':
        return "Provoked"
    elif value == "Sea Disaster":
        return "Sea Disaster"
    elif value == "Watercraft":
        return "Watercraft"
    elif value == "Boat":
        return "Watercraft"
    else:
        return "Questionable"

df2=df["Type"].apply(full_type).str.lower()


In [6]:
#Gender
df["Sex"] = df["Sex"].str.strip().str.title().replace({"Male" : "M", "M X 2" : "M", "Lli" : "M", "." : "F"})

In [7]:
#Clean Injury Column 

# 1st: formatting, cleaning spaces, lowe case and strange characters

df["Injury"] = (
    df["Injury"]
      .astype("string")
      .str.strip()
      .str.replace(r"\s+", " ", regex=True)   
      .str.lower()                             
)

#2 remove garbage like unknown, na, -, "?" without dropping "no injury":

placeholders = {"unknown","unk","n/a","na","none","null","-", "--", "—", "?", ""}
df.loc[df["Injury"].isin(placeholders), "Injury"] = pd.NA

import re

df = df.copy()

s = df["Injury"].astype("string")

# 1) detect "no injury" first
noinjury_rx = r"\bno injur(?:y|ies)\b|\buninjur(?:ed|y)\b|\bno injury to (?:\w+)\b"
noinjury = s.str.contains(noinjury_rx, na=False)

# 2) phrases that explicitly negate fatal
nonfatal_phrase = s.str.contains(r"\bnon[-\s]?fatal\b|\bnot fatal\b|\bsurvived\b", na=False)

# 3) fatal markers (only if not explicitly non-fatal)
fatal_any = s.str.contains(r"\bfatal\b|\bkilled\b|\bdied\b|\bbody not recovered\b|\bcarried off\b", na=False)
fatal_flag = (~nonfatal_phrase) & fatal_any

# 4) build the binary label
df["fatal_non_fatal"] = pd.NA
df.loc[fatal_flag, "fatal_non_fatal"] = "fatal"
df.loc[~fatal_flag & s.notna() & (s.str.len() > 0), "fatal_non_fatal"] = "non-fatal"  # includes "no injury"

In [8]:
#Activity
condition_1 = df["Activity"].str.lower().str.split().str.len() > 1
condition_2 = df["Activity"].str.replace(" ", "_")
df.loc[condition_1, "Activity"] = condition_2

In [9]:
#cleaning SPECIES column
condition_1 = df["Species "].str.lower().str.split().str.len() > 1
condition_2 = df["Species "].str.replace(" ", "_")
df.loc[condition_1, "Species "] = condition_2


In [10]:
#cleaning name column:
#format all to string type and apply correct capitalization type (lower) for better cleaning process + eliminate any special characters
df["Name"] = (df["Name"].astype("string")).str.strip().str.lower().str.replace("[()€$]", "", regex=True)
#remove innecesary titles and suffixes = 
titles = [
   "mr","mrs","ms","miss","mstr",
    "dr","drs","doctor","prof","sir","madam","madame",
    "fr","rev","pastor","pr",
    "capt","cpt","captain",
    "lt","sgt","cpl","pvt","ens","col","maj","gen","cmdr","comdr","commander","adm",
    "officer","crewman","crew","seaman","able","bosun",
    "señor","señora","señorita","sr","sra","srita","doña","don",
    "capitaine","monsieur","mademoiselle","madamoiselle",
    "capitan","capitán","ingeniero","ing","lic","arq","arquitecto"]
suffixes = [r"jr", r"sr", r"iii", r"ii", r"iv"]
removepatterns= "|".join(titles) + "|".join(suffixes) + r"|\."  #add period as well
df["Name"] = df["Name"].str.replace(removepatterns, "", regex=True)
df["Name"] = df["Name"].str.title() #Apply title case to names for better visualization

In [11]:
# Fatal cleaning
df["Fatal Y/N"]=df["Fatal Y/N"].str.strip().str.title()
df["Fatal Y/N"]=df["Fatal Y/N"].replace({"F": "Unknown", "M": "Unknown", "Nq": "Unknown", "2017": "Unknown", "Y X 2": "Unknown", "Nan": "Unknown"})

In [12]:

#AGE
df["Age"]=df["Age"].replace({"?":np.nan,"Male":np.nan,"30+":35,"Middle age":np.nan,"40?":40,"40+":40,"60+":60,"30s":35,"20/30":35,"20s":25,"!2":np.nan,"50s":55,"40s":45,"teen":15,"Teen":15,"Male":np.nan,"!6":np.nan,"!!":np.nan,"60+":65,"40?":40,"18 months":1,"Teens":15,"8 or 10":9,"\xa0 ":np.nan, " ":np.nan,"6½":7,"mid-30s":35,"16 to 18":17,
"mid-20s":25,"Ca. 33":33,"45 ":45,"9 months":0,"25 to 35":30,"25 or 28":27," '37":37,"13 or 18":16,"7 or 8":8,"Female":np.nan,"9 or 10":10,"  ":np.nan,"A.M.":np.nan,"10 or 12":11,"31 or 33":32,"2½":3,"13 or 14":14,"30 or 36":33,"MAKE LINE GREEN":np.nan,">50":55,"20's":22,"60s":65,"X":np.nan,
"(adult)":np.nan,"60's":65,"2 to 3 months":0,"middle-age":np.nan,"adult":np.nan,'"middle-age"':np.nan,"Elderly":70,"12 or 13":13,"74 ":74,"18 to 22":20,"33 or 37":35,"18 or 20":19,"a minor":16,'"young"': np.nan,"young":np.nan,"both 11":11,"?    &   14":14,"? & 19":19})
df["Age"]=df["Age"].astype(str).str.lower().str.strip()

replacement={
    "and": "&",
    "or":"&",
    ",":"&",
    
}
ages = df["Age"].astype(str).str.lower()
for old, new in replacement.items():
    ages = ages.str.replace(old, new, regex=False)
ages = ages.str.replace(r"\s+", "", regex=True)
ages=(ages.str.replace(r"\?+", "", regex=True).str.replace(r"&{2,}", "&", regex=True).str.strip("&"))
def extract_numbers(text):
    nums = re.findall(r"\d+", text)
    return [int(n) for n in nums] if nums else np.nan

extracted = ages.apply(extract_numbers)
df["Age"] = extracted.apply(lambda x: np.mean(x).round(0) if isinstance(x, list) else np.nan)
df["Age"] = df["Age"].astype("Int64")


In [13]:
#Cleaning Year Column: 
df["Year"]=(
    df["Year"].astype("string").str.strip().str.extract(r"(\d{1,4})")[0])
df["Year"]= pd.to_numeric(df["Year"], errors="coerce")

In [14]:
#Country
df["Country"] = df["Country"].str.split('/').str[0]
df["Country"] = df["Country"].str.replace(r'\?\s*\(.*\)', "", )
df["Country"] = df["Country"].str.replace('?', '', regex=False)
df["Country"] = df["Country"].str.strip().str.title()
valid_countries = [
    'United States', 'South Africa', 'Brazil', 'Maldives', 'Trinidad and Tobago',
    'Saint Kitts and Nevis', 'Saint Martin', 'United Arab Emirates', 'Sri Lanka',
    'Myanmar', 'Australia', 'Mexico', 'Costa Rica', 'Bahamas', 'Puerto Rico',
    'French Polynesia', 'Spain', 'Canary Islands', 'Vanuatu', 'Jamaica',
    'Israel', 'Philippines', 'Mozambique', 'New Caledonia', 'Egypt',
    'Thailand', 'New Zealand', 'Hawaii', 'Honduras', 'Indonesia', 'Morocco',
    'Belize', 'Cuba', 'Colombia', 'Ecuador', 'Seychelles', 'Argentina',
    'Fiji', 'England', 'Japan', 'Canada', 'Jordan', 'Papua New Guinea',
    'Reunion', 'China', 'Ireland', 'Italy', 'Malaysia', 'Libya', 'Mauritius',
    'Solomon Islands', 'Cape Verde', 'Dominican Republic', 'Cayman Islands',
    'Aruba', 'Portugal', 'Samoa', 'Kiribati', 'Taiwan', 'Palestinian Territories',
    'Guam', 'Nigeria', 'Tonga', 'Scotland', 'Croatia', 'Saudi Arabia', 'Chile',
    'Antigua', 'Kenya', 'Russia', 'South Korea', 'Malta', 'Vietnam',
    'Madagascar', 'Panama', 'Somalia', 'British Virgin Islands', 'Norway',
    'Senegal', 'Yemen', 'Liberia', 'Venezuela', 'Uruguay', 'Micronesia',
    'Slovenia', 'Curacao', 'Iceland', 'Barbados', 'Monaco', 'Guyana', 'Haiti',
    'San Domingo', 'Kuwait', 'Falkland Islands', 'Crete', 'Cyprus', 'Lebanon',
    'Paraguay', 'Georgia', 'Syria', 'Tuvalu', 'Guinea', 'Equatorial Guinea',
    'Cook Islands', 'Peru', 'Algeria', 'Ghana', 'Greenland', 'Sweden',
    'Djibouti', 'Bahrein', 'Korea'
]
def fuzzy_match_country(name):
    if pd.isnull(name):
        return name
    match, score = process.extractOne(name, valid_countries)
    if score >= 85:  
        return match
    return name  
df['Country'] = df['Country'].apply(fuzzy_match_country)

In [15]:
#State
df["State"] = df["State"].str.replace('?', '', regex=False)
df["State"] = df["State"].str.strip().str.title()

valid_states =valid_states = [
    # USA
    'California', 'Florida', 'Texas', 'New York', 'North Carolina', 'South Carolina',
    'Massachusetts', 'Georgia', 'Louisiana', 'Mississippi', 'Alabama', 'New Jersey',
    'Washington', 'Oregon', 'Utah', 'Maryland', 'Virginia', 'Delaware', 'Rhode Island',
    'Hawaii',

    # Australia
    'New South Wales', 'Queensland', 'Victoria', 'Western Australia', 'South Australia',
    'Tasmania', 'Northern Territory',

    # South Africa
    'KwaZulu-Natal', 'Eastern Cape Province', 'Western Cape Province', 'Gauteng',
    'Limpopo', 'Mpumalanga', 'Free State', 'North West', 'Northern Cape',

    # New Zealand
    'North Island', 'South Island', 'Auckland', 'Wellington', 'Canterbury', 'Otago',

    # Papua New Guinea
    'Central Province', 'Eastern Highlands', 'Western Highlands', 'Morobe', 'Madang',
    'East Sepik', 'West Sepik', 'Milne Bay', 'New Ireland', 'Bougainville',

    # Bahamas
    'New Providence', 'Grand Bahama Island', 'Abaco Islands', 'Exumas', 'Eleuthera',
    'Bimini', 'Andros Island', 'Berry Islands', 'Cat Island',

    # Brazil
    'São Paulo', 'Rio de Janeiro', 'Bahia', 'Pernambuco', 'Alagoas', 'Maranhão',
    'Santa Catarina', 'Rio Grande Do Sul',

    # Mexico
    'Quintana Roo', 'Baja California', 'Baja California Sur', 'Jalisco', 'Sonora',
    'Guerrero', 'Sinaloa', 'Tabasco', 'Tamaulipas', 'Colima',

    # Italy
    'Sardinia', 'Sicily', 'Tuscany', 'Lazio', 'Campania', 'Liguria', 'Veneto',
    'Emilia-Romagna', 'Puglia', 'Calabria',

    # Fiji
    'Viti Levu', 'Vanua Levu', 'Taveuni Island', 'Yasawa Islands', 'Kadavu Island',

    #Random cites from to 25 countries
    'Florida', 'New South Wales', 'Queensland', 'Hawaii', 'California', 'Western Australia',
    'KwaZulu-Natal', 'Western Cape Province', 'South Carolina', 'Eastern Cape Province',
    'South Australia', 'North Carolina', 'Victoria', 'Texas', 'Pernambuco', 'North Island',
    'Torres Strait', 'New Jersey', 'South Island', 'New York', 'Tasmania', 'Oregon',
    'Abaco Islands', 'Central Province', 'Northern Territory'
]


# Step 3: Fuzzy match and replace
def fuzzy_clean(x):
    if pd.isna(x):
        return np.nan
    match, score = process.extractOne(x, valid_states)
    return match if score > 85 else x

df['State_cleaned'] = df['State'].apply(fuzzy_clean)

Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: '']


In [16]:
#DAte
def date_to_year_or_year_month(df, date_col="Date", year_col="Year"):
    s = df[date_col]


    if pd.api.types.is_datetime64_any_dtype(s):
        
        df[date_col] = s.dt.strftime("%Y-%m")
        return df


    s = s.astype(str)
    s = (s.str.replace("\u00A0", " ", regex=False).str.replace("\u200b", "", regex=False).str.replace(r"[\[\]]", "", regex=True).str.replace(r"(\d{1,2})(st|nd|rd|th)\b", r"\1", regex=True).str.replace(",", "", regex=False).str.replace(r"\s+", " ", regex=True).str.strip())

    out = pd.Series(pd.NA, index=df.index, dtype="object")
    remaining = pd.Series(True, index=df.index)

    m_range = remaining & s.str.fullmatch(r"(\d{3,4})\s*[-–]\s*(\d{3,4})", na=False)
    if m_range.any():
        y12 = s[m_range].str.extract(r"(\d{3,4})\s*[-–]\s*(\d{3,4})").astype(int)
        mid = ((y12[0] + y12[1]) // 2).astype(int).astype(str)
        out.loc[m_range] = mid
        remaining &= ~m_range

 
    m_qual = remaining & s.str.fullmatch(r"(?i)(before|after|circa|about)\s+(\d{3,4})", na=False)
    if m_qual.any():
        year = s[m_qual].str.extract(r"(?i)(before|after|circa|about)\s+(\d{3,4})")[1]
        out.loc[m_qual] = year
        remaining &= ~m_qual


    m_year = remaining & s.str.fullmatch(r"\d{3,4}", na=False)
    if m_year.any():
        out.loc[m_year] = s[m_year]
        remaining &= ~m_year


    if year_col in df.columns:
        m_dm = remaining & s.str.match(r"^\d{1,2}\s+[A-Za-z]+$", na=False) & df[year_col].notna()
        if m_dm.any():
            s2 = s[m_dm] + " " + df.loc[m_dm, year_col].astype("Int64").astype(str)
            parsed_dm = pd.to_datetime(s2, errors="coerce", dayfirst=True)
            ok_idx = parsed_dm[parsed_dm.notna()].index
            out.loc[ok_idx] = parsed_dm.loc[ok_idx].dt.strftime("%Y-%m")
            remaining &= ~remaining.index.isin(ok_idx)


    if remaining.any():
        parsed = pd.to_datetime(s[remaining], errors="coerce", dayfirst=True, utc=False)
        ok_idx = parsed[parsed.notna()].index
        out.loc[ok_idx] = parsed.loc[ok_idx].dt.strftime("%Y-%m")
        remaining &= ~remaining.index.isin(ok_idx)


    if remaining.any():
        formats = ["%Y-%m-%d %H:%M:%S","%Y-%m-%d","%d/%m/%Y %H:%M:%S","%d/%m/%Y","%m/%d/%Y %H:%M:%S","%m/%d/%Y""%Y-%m-%dT%H:%M:%S","%Y-%m-%dT%H:%M:%S%z","%Y-%m-%d %H:%M:%S%z"]
        r_idx = remaining[remaining].index
        for fmt in formats:
            if not remaining.any():
                break
            try:
                parsed_fmt = pd.to_datetime(s[remaining], format=fmt, errors="coerce")
            except Exception:
                continue
            ok_idx = parsed_fmt[parsed_fmt.notna()].index
            if len(ok_idx):
                out.loc[ok_idx] = parsed_fmt.loc[ok_idx].dt.strftime("%Y-%m")
                remaining &= ~remaining.index.isin(ok_idx)

    if remaining.any():
        m_num = remaining & s.str.fullmatch(r"\d+(\.\d+)?", na=False)
        if m_num.any():
            num = pd.to_numeric(s[m_num], errors="coerce")
            parsed_num = pd.to_datetime(num, unit="D", origin="1899-12-30", errors="coerce")
            ok_idx = parsed_num[parsed_num.notna()].index
            out.loc[ok_idx] = parsed_num.loc[ok_idx].dt.strftime("%Y-%m")
            remaining &= ~remaining.index.isin(ok_idx)

    df[date_col] = out
    return df
df=date_to_year_or_year_month(df, date_col="Date", year_col="Year")

#Separate the year from the month and have them into two separate columns 

m = df["Date"].astype("string").str.strip().str.extract(
    r"^(?P<Year>\d{4})(?:-(?P<Month>\d{1,2}))?$"
)

df["Year_from_date"]  = pd.to_numeric(m["Year"], errors="coerce").astype("Int64")
df["Month_from_date"] = pd.to_numeric(m["Month"], errors="coerce").astype("Int64")

                               Selecting to 10 countries 

In [17]:
# df_selected_country = Filtering for top 10 countries with most shark attacks
df.Country.value_counts(ascending=False)
selected_countries = ["Usa", "Australia", "South Africa", "New Zealand", "Papua New Guinea", "Bahamas", "Brazil", "Mexico", "Italy", "Fiji"]
df_selected = df[df["Country"].isin(selected_countries)]

                              Groupby

In [18]:
# df_country =  groupby country 
df_country = df_selected.groupby(["Country", "Sex"]).agg(
    total_cases=("Fatal Y/N", "count"),
    fatal_yes=("Fatal Y/N", lambda x: (x == "Y").sum()),
    fatal_no=("Fatal Y/N", lambda x: (x == "N").sum()),
    avg_age=("Age", "mean"))
df_country_asc = df_country.sort_values(by="total_cases", ascending=False)

# df_country.head(10)

In [19]:
# df_state =  groupby state that has more thn 11 cases 
df_state = df_selected.groupby(["Country", "State", "Sex"]).agg(
    total_cases=("Fatal Y/N", "count"),
    fatal_yes=("Fatal Y/N", lambda x: (x == "Y").sum()),
    fatal_no=("Fatal Y/N", lambda x: (x == "N").sum()),
    avg_age=("Age", "mean"))   
df_state = df_state[df_state["total_cases"] >= 11]
df_state_asc = df_state.sort_values(by="total_cases", ascending=False)
# df_state.head(25)

                    Use the following dataframe

                        df_country =  groupby country 
                        df_country_asc =  groupby country ascending

                        df_state =  groupby state that has more thn 11 cases 
                        df_state_asc =  groupby state that has more thn 11 cases ascending

In [20]:
# Calculations from df_country
overall_avg_age = df_country["avg_age"].mean()

total_males = df_country.loc[df_country.index.get_level_values("Sex") == "M", "total_cases"].sum()
total_females = df_country.loc[df_country.index.get_level_values("Sex") == "F", "total_cases"].sum()

fatal_males = df_country.loc[df_country.index.get_level_values("Sex") == "M", "fatal_yes"].sum()
fatal_females = df_country.loc[df_country.index.get_level_values("Sex") == "F", "fatal_yes"].sum()

total_cases_all = df_country["total_cases"].sum()
total_fatal_yes = df_country["fatal_yes"].sum()
total_fatal_no = df_country["fatal_no"].sum()

fatality_rate_males = (fatal_males / total_males) * 100
fatality_rate_females = (fatal_females / total_females) * 100
overall_fatality_rate = (total_fatal_yes / total_cases_all) * 100

print("Statistics for df_country:")
print("*" *30)
print("Overall avg age:", overall_avg_age)
print("Total males:", total_males)
print("Total females:", total_females)
print("Total cases:", total_cases_all)
print("Total fatal yes:", total_fatal_yes)
print("Total fatal no:", total_fatal_no)
print("Male percentage of total cases:", (total_males / total_cases_all) * 100)
print("Female percentage of total cases:", (total_females / total_cases_all) * 100)
print("Male fatality rate (%):", fatality_rate_males)
print("Female fatality rate (%):", fatality_rate_females)
print("Overall fatality rate (%):", overall_fatality_rate)

Statistics for df_country:
******************************
Overall avg age: 27.31740744588467
Total males: 4107
Total females: 604
Total cases: 4712
Total fatal yes: 822
Total fatal no: 3859
Male percentage of total cases: 87.16044142614601
Female percentage of total cases: 12.818336162988114
Male fatality rate (%): 18.407596785975162
Female fatality rate (%): 10.927152317880795
Overall fatality rate (%): 17.44482173174873


In [21]:
# df_country.head(10)

In [22]:
# df_country_asc.head(10)

In [23]:
# df_state.head(10)


In [24]:
# df_state_asc.head(10)